In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [2]:
import numpy as np
import pandas as pd

In [3]:
# prompt: import final_dataset.csv from google drive

#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
# Define the chunk size
chunk_size = 100000

# Initialize an empty list to store the sample
sample_list = []

# Read the CSV in chunks without early termination
for chunk in pd.read_csv('/content/drive/My Drive/final_dataset.csv', chunksize=chunk_size):
    # Sample from the current chunk
    sample = chunk.sample(frac=0.1, random_state=42)
    sample_list.append(sample)

# Concatenate all sampled chunks
df = pd.concat(sample_list, ignore_index=True)
print(df['Label'].value_counts())
print(df['Protocol'].value_counts())

Label
ddos      647297
Benign    632166
Name: count, dtype: int64
Protocol
6     1049110
17     218954
0       11399
Name: count, dtype: int64


In [5]:
df.columns

Index(['Unnamed: 0', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port',
       'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag

In [6]:

df = df.dropna()

In [7]:
features_to_keep = [
    'Src Port', 'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts',
    'Tot Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean',
    'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
    'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Fwd Pkts/s',
    'Bwd Pkts/s', 'Down/Up Ratio', 'FIN Flag Cnt', 'SYN Flag Cnt',
    'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
    'CWE Flag Count', 'ECE Flag Cnt', 'Subflow Fwd Pkts', 'Subflow Bwd Pkts',
    'Subflow Fwd Byts', 'Subflow Bwd Byts', 'Active Mean', 'Active Std', 'Active Max',
    'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label'
]
df = df[features_to_keep]

In [8]:
import numpy as np

# Check for infinite values
inf_mask = np.isinf(df.select_dtypes(include=[np.number]))
inf_columns = inf_mask.any()
print("Columns with inf values:\n", inf_columns[inf_columns].index.tolist())
df_clean = df.replace([np.inf, -np.inf], np.nan).dropna()

Columns with inf values:
 ['Flow Byts/s', 'Flow Pkts/s']


In [9]:
print("Remaining inf:", np.isinf(df_clean.select_dtypes(include=[np.number])).any().any())
print("Remaining NaN:", df_clean.isna().any().any())

Remaining inf: False
Remaining NaN: False


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1276490 entries, 0 to 1279462
Data columns (total 40 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Src Port          1276490 non-null  int64  
 1   Dst Port          1276490 non-null  int64  
 2   Protocol          1276490 non-null  int64  
 3   Flow Duration     1276490 non-null  int64  
 4   Tot Fwd Pkts      1276490 non-null  int64  
 5   Tot Bwd Pkts      1276490 non-null  int64  
 6   Fwd Pkt Len Max   1276490 non-null  float64
 7   Fwd Pkt Len Min   1276490 non-null  float64
 8   Fwd Pkt Len Mean  1276490 non-null  float64
 9   Fwd Pkt Len Std   1276490 non-null  float64
 10  Bwd Pkt Len Max   1276490 non-null  float64
 11  Bwd Pkt Len Min   1276490 non-null  float64
 12  Bwd Pkt Len Mean  1276490 non-null  float64
 13  Bwd Pkt Len Std   1276490 non-null  float64
 14  Flow Byts/s       1276490 non-null  float64
 15  Flow Pkts/s       1276490 non-null  float64
 16  Fwd P

In [11]:
df.head()

,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Subflow Bwd Byts,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,34408,80,6,5032233,3,5,603.0,0.0,201.000000,348.142212,...,972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
1,40676,80,6,10993127,3,5,408.0,0.0,136.000000,235.558910,...,972,783.0,0.0,783.0,783.0,5990709.0,0.0,5990709.0,5990709.0,ddos
2,4020,80,6,479425,3,7,142.0,0.0,47.333333,81.983738,...,4702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
3,36102,80,6,9042123,1,1,0.0,0.0,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,9042123.0,0.0,9042123.0,9042123.0,ddos
4,44284,80,6,1907579,1,1,0.0,0.0,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos


In [12]:
X = df_clean.drop('Label', axis=1)
y = df_clean['Label']

In [13]:
df.columns

Index(['Src Port', 'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Down/Up Ratio',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Subflow Fwd Pkts', 'Subflow Bwd Pkts', 'Subflow Fwd Byts',
       'Subflow Bwd Byts', 'Active Mean', 'Active Std', 'Active Max',
       'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label'],
      dtype='object')

In [14]:
rows, columns = df.shape
print(f"Rows: {rows}, Columns: {columns}")

Rows: 1276490, Columns: 40


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy:",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

      Benign       1.00      1.00      1.00    125483
        ddos       1.00      1.00      1.00    129459

    accuracy                           1.00    254942
   macro avg       1.00      1.00      1.00    254942
weighted avg       1.00      1.00      1.00    254942

Confusion Matrix:
 [[125341    142]
 [    98 129361]]
Accuracy: 0.999058609409199


In [18]:
import joblib
joblib.dump(model, '/content/drive/My Drive/model_final_FINAL.joblib')

['/content/drive/My Drive/model_final_FINAL.joblib']

In [19]:
test = model.predict([[57090,80,6,23.674991,107,67,519.0,10.0,297.64486,217.633379,776.0,0.0,291.313433,146.091628,2169.631246,7.349528,4.519537,2.829991,0.626168,6,0,0,73,174,0,0,0,8.916667,5.583333,2654.0,1626.5,23674990.89241,0.0,23674990.89241,23674990.89241,0.0,0.0,0.0,0.0]])

In [20]:
print(test)

['Benign']
